looking into the titles of individual articles with certain features

In [1]:
import logging
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus, textcorpus
import numpy as np
from gensim.matutils import hellinger
import time
import pickle
import pyLDAvis
import matplotlib.pyplot as plt
import os
from scipy.stats import entropy

from IPython.core.debugger import set_trace


In [2]:
alldata_new = pickle.load(open('output/dtm_processed_output.p', 'rb'))
alldata_new.keys()

dict_keys(['terms', 'docnames', 'term_topic', 'doc_topic', 'docs_per_year'])

In [3]:
# load data
doc_year=alldata_new['docs_per_year']
doc_ids =[0]+list(np.cumsum(doc_year))

term_topic = alldata_new['term_topic']# term_topic is n_years*n_topics*n_terms
terms = alldata_new['terms']

In [8]:
doc_topicyrs = alldata_new['doc_topic']

doc_topic = []
for year in range(len(term_topic)):    
    doc_topic.append(alldata_new['doc_topic'][doc_ids[year]:doc_ids[year+1]])# doc_topic is nyear*n_docs given year*n_topics
    
# rename topics by their top freq word
topics = range(term_topic.shape[1])

def topic_label(topic, term_topic, terms):
    term_freqs = np.sum(term_topic[:,topic,:], axis = 0)
    max_term = np.argsort(-term_freqs)[0]
    return(terms[max_term])


topic_labels = [topic_label(topic, term_topic, terms) for topic in topics]
print(topic_labels)
"""with open('output/all_visdtm.p','br') as f:
    allvisdtm=pickle.load(f)
for visdtm in allvisdtm:

    visdtm[0]['topiclabel']=topic_labels"""
#topic_labels

['sequence', 'face', 'reasoning', 'text', 'color', 'decision', 'sentence', 'causal', 'concept', 'child', 'spatial', 'error', 'category', 'english', 'student', 'network', 'speaker', 'probability', 'agent', 'visual']


"with open('output/all_visdtm.p','br') as f:\n    allvisdtm=pickle.load(f)\nfor visdtm in allvisdtm:\n\n    visdtm[0]['topiclabel']=topic_labels"

In [18]:
doc_ids

[0,
 159,
 347,
 492,
 693,
 937,
 1334,
 1711,
 1980,
 2153,
 2667,
 3110,
 3663,
 4094,
 4770,
 5267,
 5864,
 6328,
 6920]

In [20]:
alltitles=alldata_new['docnames']
doctitle = []
for year in range(len(doc_year)):
    doctitle.append(alltitles[doc_ids[year]:doc_ids[year+1]])

In [23]:
# feature1: most typical article of each topics
with open('highest_freq.txt','w') as f:
    for kt in range(len(topic_labels)):
        f.write('\ntopic '+str(kt)+': '+topic_labels[kt]+'\n')        
        for year in range(len(doc_topic)):  
            topicfreq=np.array(doc_topic[year]).T[kt]
            idx=np.argmax(topicfreq)
            title=doctitle[year][idx]
            
            f.write(str(year+2000)+': '+title+'\n')

In [ ]:
# feature2: "purest" / "most chaotic paper" 
maxen=0
with open('maxmin_entropy.txt','w') as f:
    for year in range(len(doc_topic)):  
        f.writelines('\n'+str(year+2000)+':\n')
        alldocs = doc_topic[year]
        allentrop=[]
        for d in alldocs:
            allentrop.append(entropy(d))
        # rank them
        maxE = np.argmax(allentrop)
        minE = np.argmin(allentrop)
        f.writelines('max:'+gettitle(year,maxE)+'\n')
        f.writelines('min:'+gettitle(year,minE)+'\n')
        
        # max entropy across years?
        if maxE>maxen:
            maxen=maxE
            maxtitle=gettitle(year,maxE)
print(maxtitle)

In [ ]:
# does entropy correlate with length of title?
alltitlen=[]
allentrop=[]
for year in range(len(doc_topic)):  
    alldocs = doc_topic[year]
    for d in alldocs:
        allentrop.append(entropy(d))
    for idx in range(len(alldocs)):
        alltitlen.append(len(gettitle(year,idx)))
plt.plot(alltitlen,allentrop,'.')
plt.show()



In [ ]:
for kt in range(len(topic_labels)):
    print(len(doc))